# Advanced Querying Mongo

Importing libraries and setting up connection

In [8]:
%pip install pymongo
from pymongo import MongoClient

# Crear una instancia del cliente de MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Reemplaza con la URL de tu servidor MongoDB

# Acceder a la base de datos y la colección
db = client.Companies
colections = db.Colections
#MUY IMPORTANTE ESCRIBIRLO EXACTAMENTE IGUAL, MAYUSCULAS Y MINUSCULAS


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [37]:
query={'name': 'Babelgum'}
select = {'_id': 0, 'name': 1}
result = list(colections.find(query, select).limit(5))
result


[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [38]:
query={'number_of_employees': {'$gt': 5000}} #query es lo que quieres
select= {'_id': 0, 'name': 1, 'number_of_employees' : 1} #que es lo que te va a dar
result = list(colections.find(query, select).limit(20))
result




[{'name': 'Facebook', 'number_of_employees': 5299},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Cisco', 'number_of_employees': 63000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'Google', 'number_of_employees': 28000},
 {'name': 'Intel', 'number_of_employees': 86300},
 {'name': 'Nintendo', 'number_of_employees': 5080},
 {'name': 'Adobe Systems', 'number_of_employees': 7173},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'The Walt Disney Company', 'number_of_employees': 25000},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Sun Microsystems', 'number_of_employees': 33350},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Microsoft', 'number_of_employees': 90000},
 {'name': 'Apple', 'number_of_employees': 80000},
 {'name': 'NetApp', 'number_of_employees': 8000},
 {

In [42]:
query = {'number_of_employees': {'$gt': 5000}}
#select= {'_id': 0, 'name': 1, 'number_of_employees' : 1} # RECUERDA QUE AQUI TAMBN IRIRA PERO NO LO PONGO XQ ESTA DECLARADO ARRIBA
result = list(colections.find(query, select).sort('number_of_employees', 1).limit(20))
result

#lo mismo pero diferente, PARA IRLO ENTENDIENDO

[{'name': 'Nintendo', 'number_of_employees': 5080},
 {'name': 'Hexaware Technologies', 'number_of_employees': 5200},
 {'name': 'Facebook', 'number_of_employees': 5299},
 {'name': 'OpenText', 'number_of_employees': 5300},
 {'name': 'CPM Braxis', 'number_of_employees': 5400},
 {'name': 'LSI', 'number_of_employees': 5400},
 {'name': 'Microchip Technologies', 'number_of_employees': 5500},
 {'name': 'Mediaset', 'number_of_employees': 5729},
 {'name': 'Mindray Medical International', 'number_of_employees': 5763},
 {'name': 'Dentsu', 'number_of_employees': 6000},
 {'name': 'Atmel', 'number_of_employees': 6000},
 {'name': 'Tata Communications', 'number_of_employees': 6000},
 {'name': 'Baidu', 'number_of_employees': 6000},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'SRA International', 'number_of_employees': 6400},
 {'name': "Moody's", 'number_of_employees': 6800},
 {'name': 'ManTech', 'number_of_employees': 7000},
 {'name': 'Ad

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [43]:
query={'$and': [{'founded_year': {'$gte': 2000}},
                {'founded_year': {'$lte': 2005}}]}
                
select= {'_id': 0, 'name': 1, 'founded_year' : 1}
result = list(colections.find(query, select).limit(5))
result



[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [ ]:
# Your Code

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [ ]:
# Your Code

### 6. All the companies that don't include the `partners` field.

In [ ]:
# Your Code

### 7. All the companies that have a null type of value on the `category_code` field.

In [ ]:
# Your Code

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [ ]:
# Your Code

### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
# Your Code

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
# Your Code

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
# Your Code

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code